In [2]:
import pandas as pd
import requests
import plotly.express as px

In [8]:
# Use the correct base URL for Binance Futures API (fapi.binance.com)
url = f"https://fapi.binance.com/fapi/v1/klines"
params = {
    "symbol": "SOLUSDT",
    "interval": "1m",
    "limit": 100
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    data = response.json()
    print(data)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    if 'response' in locals():
        print(f"Response text: {response.text}")

[[1764523200000, '138.8300', '138.8400', '138.7100', '138.7900', '9478.57', 1764523259999, '1315477.924800', 1316, '3547.30', '492278.034000', '0'], [1764523260000, '138.7900', '138.7900', '138.6400', '138.6400', '3854.70', 1764523319999, '534685.873200', 858, '733.72', '101783.342700', '0'], [1764523320000, '138.6400', '138.8100', '138.6200', '138.7600', '5425.41', 1764523379999, '752656.432400', 1105, '3736.08', '518341.622900', '0'], [1764523380000, '138.7600', '138.7800', '138.7200', '138.7600', '2991.16', 1764523439999, '415044.028100', 723, '1432.45', '198763.415400', '0'], [1764523440000, '138.7600', '138.8200', '138.6500', '138.7100', '5593.91', 1764523499999, '775871.281500', 1229, '2593.17', '359656.878700', '0'], [1764523500000, '138.7100', '138.7100', '138.5900', '138.6000', '5000.76', 1764523559999, '693277.514600', 940, '1049.24', '145464.782200', '0'], [1764523560000, '138.6000', '138.7900', '138.5100', '138.6700', '26628.27', 1764523619999, '3692512.726400', 2632, '1762

In [11]:
df_raw = pd.DataFrame(data)
df_raw.columns = [
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
]
df_raw

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1764523200000,138.8300,138.8400,138.7100,138.7900,9478.57,1764523259999,1315477.924800,1316,3547.30,492278.034000,0
1,1764523260000,138.7900,138.7900,138.6400,138.6400,3854.70,1764523319999,534685.873200,858,733.72,101783.342700,0
2,1764523320000,138.6400,138.8100,138.6200,138.7600,5425.41,1764523379999,752656.432400,1105,3736.08,518341.622900,0
3,1764523380000,138.7600,138.7800,138.7200,138.7600,2991.16,1764523439999,415044.028100,723,1432.45,198763.415400,0
4,1764523440000,138.7600,138.8200,138.6500,138.7100,5593.91,1764523499999,775871.281500,1229,2593.17,359656.878700,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1764528900000,138.4600,138.5100,138.4500,138.5100,4958.21,1764528959999,686542.723900,610,3246.94,449590.631700,0
96,1764528960000,138.5100,138.5600,138.5100,138.5600,5441.33,1764529019999,753812.696400,611,4588.92,635725.242600,0
97,1764529020000,138.5600,138.5800,138.5300,138.5300,3147.49,1764529079999,436126.411600,443,1587.71,220001.549600,0
98,1764529080000,138.5300,138.5700,138.5300,138.5600,1964.09,1764529139999,272124.359300,364,1115.30,154521.562700,0


In [14]:
# --- DataFrame 1: Opening Data ---
df_open = df_raw[['open_time', 'open']].copy() 
df_open.columns = ['opening_time', 'opening_price']

# 1. Convert 'opening_time' from Unix timestamp (ms) to readable datetime objects
# 2. Convert 'opening_price' from string to float for numerical analysis
df_open['opening_time'] = pd.to_datetime(df_open['opening_time'], unit='ms')
df_open['opening_price'] = df_open['opening_price'].astype(float)
df_open

,opening_time,opening_price
0,2025-11-30 17:20:00,138.83
1,2025-11-30 17:21:00,138.79
2,2025-11-30 17:22:00,138.64
3,2025-11-30 17:23:00,138.76
4,2025-11-30 17:24:00,138.76
...,...,...
95,2025-11-30 18:55:00,138.46
96,2025-11-30 18:56:00,138.51
97,2025-11-30 18:57:00,138.56
98,2025-11-30 18:58:00,138.53


In [15]:
# --- DataFrame 2: Closing Data ---
df_close = df_raw[['close_time', 'close']].copy()
df_close.columns = ['closing_time', 'closing_price']

# Data Type Conversion:
# 1. Convert 'closing_time' from Unix timestamp (ms) to readable datetime objects
# 2. Convert 'closing_price' from string to float
df_close['closing_time'] = pd.to_datetime(df_close['closing_time'], unit='ms')
df_close['closing_price'] = df_close['closing_price'].astype(float)
df_close

,closing_time,closing_price
0,2025-11-30 17:20:59.999,138.79
1,2025-11-30 17:21:59.999,138.64
2,2025-11-30 17:22:59.999,138.76
3,2025-11-30 17:23:59.999,138.76
4,2025-11-30 17:24:59.999,138.71
...,...,...
95,2025-11-30 18:55:59.999,138.51
96,2025-11-30 18:56:59.999,138.56
97,2025-11-30 18:57:59.999,138.53
98,2025-11-30 18:58:59.999,138.56
